In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

def main():
    data = pd.read_csv("Face-Feat.txt")
    print(data)
    labels = data["Emotion"]
    inputs = data.drop("Emotion", axis = 1)
    data_in, test_in, data_out, test_out = train_test_split(inputs, labels, test_size=0.1, stratify=labels)
    train_in, val_in, train_out, val_out = train_test_split(data_in, data_out, test_size = 0.2/0.9, stratify = data_out)
    model = SVC()
    model.fit(train_in, train_out)
    output = model.predict(val_in)


if __name__ == "__main__":
    main


ModuleNotFoundError: No module named 'sklearn'